In [ ]:
import os
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

In [ ]:
dfB = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/data/midterm/drive_stats_2019_Q1/2019-01-0[1-7].csv")

In [ ]:
#dfB.describe()

In [ ]:
dfA0 = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/data/midterm/drive_stats_2019_Q1/2019-01-0[8-9].csv")

In [ ]:
dfA1 = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/data/midterm/drive_stats_2019_Q1/2019-01-[1-3][0-9].csv")

In [ ]:
dfA2 = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/data/midterm/drive_stats_2019_Q1/2019-02-[0-2][0-9].csv")

In [ ]:
dfA3 = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/data/midterm/drive_stats_2019_Q1/2019-03-[0-3][0-9].csv")

In [ ]:
dfAFinal = dfA0.union(dfA1).union(dfA2).union(dfA3)

In [ ]:
#dfAFinal.count()

In [ ]:
import pybloom_live

In [ ]:
bloom = pybloom_live.BloomFilter(capacity=dfB.count(), error_rate=0.001)

In [ ]:
dfB_Models = dfB.select('model')
dfB_Models = dfB_Models.drop_duplicates(["model"])

#dfB_Models.show(10)

In [ ]:
[bloom.add(x) for x in dfB_Models.rdd.toLocalIterator()]

In [ ]:
all([(x in bloom) for x in dfB_Models.rdd.toLocalIterator()])

In [ ]:
broadcastFilter = sc.broadcast(bloom)

In [ ]:
x = dfAFinal.rdd.filter(lambda x : broadcastFilter.value.mightContain(x(0)))

In [ ]:
#i tried the below methods but i just wasnt able to get it running so i went with sql as an easy option i know its know distrubted 

In [ ]:
#dfAFinal.join(bloom).where(dfAFinal.select('model') in bloom)

In [ ]:
#x = dfAFinal.select('model').rdd.filter(lambda x : broadcastFilter.value.mightContain(x(0)))

In [ ]:
#x.take(10)